In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import os
from cleantext import clean

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
from pandas import DataFrame

In [3]:
arr = os.listdir('.')
print(arr)

['Label_train_2.csv', '.DS_Store', 'Creation_matrice_Creation.ipynb', 'FinalDBelon.csv', 'Data_train.csv', 'DataNONEL_clean.csv', 'Clean_Tweets_EM.ipynb', 'Creation_matrice_Detection.ipynb', 'README.md', 'DataBase_temporary_creation.csv', 'Tweets_Not_ElonMusk.csv', 'DataBase.csv', '.gitignore', 'Reseau_neurone.ipynb', 'Data_train_2.csv', 'DataElon_clean.csv', 'Vectorisation.ipynb', '.ipynb_checkpoints', 'Label_train.csv', 'Clean_Tweets_NONEM.ipynb', '.git', 'TweetsElonMusk.csv']


In [4]:
assert hasattr(tf, 'function')

In [5]:
Data_train = pd.read_csv("Data_train.csv")
Label_train = pd.read_csv("Label_train.csv")

In [6]:
def Construction_modele(Data_train, Label_train):
    #Construction couche de neurones
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(128, activation="relu"))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))
    #Mise au bon format des donnes pour le modele
    Tweet_1 = Data_train.values
    Tweet_2 = Tweet_1.astype(dtype='float32')
    Label_1 = Label_train.values
    Label_2 = Label_1[:,0].astype(dtype='uint8')
    #Ajustement modele et choix de metrique
    model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
    #Entrainement du modele
    model.fit(Tweet_2, Label_2, epochs=18)
    return model

In [7]:
modelll = Construction_modele(Data_train, Label_train)

Epoch 1/18
348/348 [==============================] - 1s 1ms/step - loss: 0.6890 - accuracy: 0.5366
Epoch 2/18
348/348 [==============================] - 0s 1ms/step - loss: 0.6721 - accuracy: 0.6668
Epoch 3/18
348/348 [==============================] - 0s 1ms/step - loss: 0.6444 - accuracy: 0.7264
Epoch 4/18
348/348 [==============================] - 0s 1ms/step - loss: 0.5994 - accuracy: 0.7561
Epoch 5/18
348/348 [==============================] - 0s 1ms/step - loss: 0.5390 - accuracy: 0.7795
Epoch 6/18
348/348 [==============================] - 0s 1ms/step - loss: 0.4779 - accuracy: 0.8016
Epoch 7/18
348/348 [==============================] - 0s 1ms/step - loss: 0.4283 - accuracy: 0.8205
Epoch 8/18
348/348 [==============================] - 0s 1ms/step - loss: 0.3915 - accuracy: 0.8338
Epoch 9/18
348/348 [==============================] - 0s 1ms/step - loss: 0.3642 - accuracy: 0.8459
Epoch 10/18
348/348 [==============================] - 0s 1ms/step - loss: 0.3423 - accuracy: 0.8543

In [8]:
def Vectorisation(Data_train, tweet):   #prend un tweet en argument sous la forme d'une string et renvoi son vecteur sous forme d'une database dans l espace vectoriel de la matrice
    Vecteurs = Data_train.columns
    row = []
    
    dictionnary = dict()
    for mot in Vecteurs:
        dictionnary[mot] = 0
    
    Mot_tweet = tweet.split(" ")
    
    for index, mot in enumerate(Mot_tweet):
            if mot in Vecteurs:
                dictionnary[mot] = index + 1
                
    row.append(dictionnary)
    Data_out = DataFrame(row)
    
    if Data_out.loc[0].max() != 0:
        Data_out.loc[0] = Data_out.loc[0] / Data_out.loc[0].max()
    
    return Data_out
    
    
    

La fonction detection prend en argument un tweet sous la forme d'un string, le reseau de neurone entraine et la base de donnee d entrainement utilise pour vectorizer le tweet. La fonction renvoi un tuple de la forme
\
\
("Elon Musk detecte a x%", 1)
\
("Indetermine", 2)
\
("Pas Elon Musk a x%", 0)
\
\
1 est renvoye ssi x >= 60%
\
0 est renvoye ssi x >= 60%
\
2 est renvoye ssi la probabilite que ce soit Elon Musk (ou non) soit comprise entre 40% et 60%

In [9]:
def detection(tweet_sentence, model_neurone, Data_train):
    tweet_sentence = tweet_sentence.lower()
    tweet_sentence = clean(tweet_sentence, no_emoji=True, no_punct=True, no_urls = True, replace_with_url='', no_digits=True, replace_with_digit='', no_currency_symbols=True, replace_with_currency_symbol='')
    vecteur_phrase = Vectorisation(Data_train, tweet_sentence)
    vecteur = vecteur_phrase.values
    vecteur = vecteur.astype(dtype='float32')
    if vecteur.sum() == 0:
        char = "Indetermine"
        return (char, 2)
    Prediction = model_neurone.predict(vecteur)
    if Prediction[0, 1] >= 0.6:
        char = "Elon Musk detecte a " + str(Prediction[0, 1]) + "%"
        return (char, 1)
    elif Prediction[0,1] >= 0.4:
        char = "Indetermine"
        return (char, 2)
    else :
        char = "Pas Elon Musk a " + str(Prediction[0, 0]) + "%"
        return (char, 0)

In [ ]:
def detection_apprentissage(tweet_sentence, model_neurone, Data_train):
    

In [24]:
phrase = 'I sing every morning in my shower, i love space and tesla and bitcoin'
detection(phrase, modelll, Data_train)

('Elon Musk detecte a 0.91566247%', 1)